# LIBRARIES

In [1]:
import pandas as pd
import numpy as np
from pkg.constants import *
from pkg.greedy import greedy

# LECTURA DE DATOS

In [2]:
index_df    = pd.read_csv(CERCANAS_INDICES_PATH) #df: dataframe
distance_df = pd.read_csv(CERCANAS_KMS_PATH)
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 

In [3]:
display(index_df)
display(distance_df)
display(deltas_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,7,4,6,10,8,1,12,15,11,5,2,9,3,14,13
1,1,12,6,10,5,7,0,15,8,11,4,2,9,3,14,13
2,2,9,11,15,3,5,12,8,6,1,7,0,10,4,14,13
3,3,9,2,11,5,15,12,1,6,8,14,7,10,13,0,4
4,4,0,7,8,6,10,15,1,12,11,5,2,9,3,14,13
5,5,12,1,11,6,15,2,9,8,10,7,3,0,4,14,13
6,6,1,12,7,10,0,8,5,15,11,4,2,9,3,14,13
7,7,0,10,6,1,4,8,12,15,5,11,2,9,3,14,13
8,8,15,6,0,11,7,12,4,1,10,5,2,9,3,14,13
9,9,3,2,11,5,15,12,1,6,8,7,10,0,4,14,13


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.461228,0.716557,0.931362,0.947728,1.063519,1.421298,1.558761,1.675331,2.018937,2.043420,2.788769,3.209743,3.837960,7.175932,7.586370
1,0.0,0.553019,0.593921,0.819978,0.921034,1.004535,1.421298,1.511249,1.579138,1.590787,2.134860,2.173151,2.318746,2.928694,6.035335,6.457200
2,0.0,0.661118,0.771991,1.149044,1.160921,1.347852,1.623997,1.906423,2.075763,2.173151,2.676229,2.788769,2.845542,3.238744,4.584395,4.969495
3,0.0,0.631582,1.160921,1.883933,2.007667,2.278177,2.443575,2.928694,3.017079,3.041827,3.425126,3.648143,3.707595,3.813989,3.837960,4.350875
4,0.0,0.716557,1.168679,1.344407,1.614828,1.618877,2.089736,2.134860,2.220537,2.474299,2.690724,3.238744,3.736570,4.350875,7.742114,8.145095
5,0.0,0.489722,0.921034,1.003747,1.122053,1.164393,1.347852,1.401056,1.667223,1.713638,1.741507,2.007667,2.043420,2.690724,5.176668,5.594235
6,0.0,0.593921,0.632883,0.633576,0.798741,0.931362,1.013820,1.122053,1.147641,1.362298,1.614828,2.075763,2.385985,3.017079,6.287533,6.702453
7,0.0,0.461228,0.507316,0.633576,1.004535,1.168679,1.219975,1.252991,1.648698,1.741507,1.932000,2.676229,3.016684,3.648143,6.915778,7.331807
8,0.0,0.763651,1.013820,1.063519,1.158661,1.219975,1.319393,1.344407,1.579138,1.658434,1.667223,1.906423,2.445856,3.041827,6.458258,6.853230
9,0.0,0.631582,0.661118,1.289733,1.401056,1.685564,1.819566,2.318746,2.385985,2.445856,3.016684,3.085136,3.209743,3.736570,4.013323,4.411380


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,5,7,13,6,8,13,8,9,6,10,10,18,8,13,15,14
1,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
474,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
475,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
476,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,0,0


# PREPROCESAMIENTO DE DATOS

In [4]:
index_matrix    = index_df.to_numpy()
distance_matrix = distance_df.to_numpy()
deltas_matrix   = deltas_df.to_numpy()

# SOLUCIÓN GREEDY

In [5]:
initial_state = deltas_matrix[0,:]
greedy_sol = greedy(initial_state)
greedy_sol.sum()

220.0

# FUNCION DE COSTES